In [1]:
# import numpy as np
# import fitsio
# import matplotlib.pyplot as plt
# from matplotlib.colors import LogNorm

In [2]:
# matchedFile = '/afs/hep.wisc.edu/home/kkboone/data/temp/Balrog_Matched_Filtered.fits'
# ssiFile = '/afs/hep.wisc.edu/home/kkboone/data/perm/InjectionsWithinTiles.fits'

In [3]:
# matchedData = fitsio.read(matchedFile, columns = ['meas_EXTENDED_CLASS_SOF', 'true_ra', 'true_dec', 'true_bdf_T'])
# ssiData = fitsio.read(ssiFile, columns = ['meas_EXTENDED_CLASS_SOF', 'true_ra', 'true_dec'])

In [4]:
# plt.hist(matchedData['meas_EXTENDED_CLASS_SOF'], bins = np.linspace(-0.5, 3.5, 5))

In [5]:
# star0Indices = np.where(matchedData['meas_EXTENDED_CLASS_SOF'] == 0)[0]
# star1Indices = np.where(matchedData['meas_EXTENDED_CLASS_SOF'] == 1)[0]
# galaxy2Indices = np.where(matchedData['meas_EXTENDED_CLASS_SOF'] == 2)[0]
# galaxy3Indices = np.where(matchedData['meas_EXTENDED_CLASS_SOF'] == 3)[0]

In [6]:
# star0BDF = matchedData['true_bdf_T'][star0Indices]
# star1BDF = matchedData['true_bdf_T'][star1Indices]
# galaxy2BDF = matchedData['true_bdf_T'][galaxy2Indices]
# galaxy3BDF = matchedData['true_bdf_T'][galaxy3Indices]

# star0RA = matchedData['true_ra'][star0Indices]
# star1BDF = matchedData['true_bdf_T'][star1Indices]
# galaxy2BDF = matchedData['true_bdf_T'][galaxy2Indices]
# galaxy3RA = matchedData['true_ra'][galaxy3Indices]

# star0DEC = matchedData['true_dec'][star0Indices]
# star1BDF = matchedData['true_bdf_T'][star1Indices]
# galaxy2BDF = matchedData['true_bdf_T'][galaxy2Indices]
# galaxy3DEC = matchedData['true_dec'][galaxy3Indices]

In [7]:
# plt.figure(dpi=150)
# plt.hist(star0BDF, bins = np.linspace(0, 1, 100), edgecolor='black', linewidth=.2)
# plt.show()

In [8]:
# plt.figure(dpi=150)
# plt.hist(star1BDF, bins = np.linspace(0, 1, 100), color = 'g', edgecolor='black', linewidth=.2)
# plt.show()

In [9]:
# plt.figure(dpi=150)
# plt.hist(galaxy2BDF, bins = np.linspace(0, 1, 100), color = 'y', edgecolor='black', linewidth=.2)
# plt.show()

In [10]:
# plt.figure(dpi=150)
# plt.hist(galaxy3BDF, bins = np.linspace(0, 1, 100), color = 'r', edgecolor='black', linewidth=.2)
# plt.show()

In [11]:
#plt.figure(dpi=150)
#plt.hist2d(galaxyRA, galaxyDEC, 
#           bins = (np.linspace(21.85,23.1,100, endpoint=True), np.linspace(-53.2,-52.42,100, endpoint=True)),
#           cmap=plt.cm.Blues)
#plt.colorbar()
#plt.show()

In [12]:
# len(star0BDF)*100

In [13]:
import numpy as np
import fitsio
import healpy as hp

In [14]:
starFile = '/afs/hep.wisc.edu/bechtol-group/MegansThings/balrog_detection_catalog_sof_run2_stars_v1.4_avg_added_match_flags.fits'
# This reads in all of the data. Most of these are just flags, the only pieces that get used much outside
# of filtering are true_ra and true_dec which get used to convert into healPixels.
starData = fitsio.read(starFile, columns = ['detected', 'true_ra', 'true_dec',
                                            'flags_foreground', 'flags_badregions', 'flags_footprint',
                                            'meas_FLAGS_GOLD_SOF_ONLY', 'match_flag_1.5_asec'])

In [15]:
# These are in degrees which is why lonlat is set to True in the next cell.
RA = starData['true_ra']
DEC = starData['true_dec']
# Everything from here on out is simply used in order to filter the data
DETECTED = starData['detected'] # This specifically is used for now to filter out any non detected injection.
FOREGROUND = starData['flags_foreground']
BADREGIONS = starData['flags_badregions']
FOOTPRINT = starData['flags_footprint']
GOLDSOF = starData['meas_FLAGS_GOLD_SOF_ONLY']
ARCSECONDS = starData['match_flag_1.5_asec']
# i_mag = starData['meas_cm_mag_deredden'][:,2] This gets used if we filter at all based on i magnitude.

# This is used to filter out any injections that either weren't detected or had flags raised.
cutIndices = np.where((FOREGROUND == 0) & 
                      (BADREGIONS < 2) & 
                      (FOOTPRINT == 1) & 
                      # (GOLDSOF < 2) &
                      (ARCSECONDS < 2))[0]

aveEff = np.sum(DETECTED) / len(DETECTED)

DETECTED = DETECTED[cutIndices]
RA = RA[cutIndices]
DEC = DEC[cutIndices]

print(aveEff)

0.7282926019070762


In [16]:
res = 2048
# This converts the RA and DEC values from above to healpixels so we can compare to the sky condition.
starPixels = hp.ang2pix(res, RA, DEC, lonlat = True, nest = True)

In [17]:
# Here I consider a valid pixel any pixel that has a non "bad value" for the condition and fracDet 
# and that has at least one of the filtered stars in it.
pix = fitsio.read('/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/valid_pixel_2048_conditions.fits')['PIX_2048']
validPixels = np.full(12*(res**2), False, dtype = bool)
validPixels[pix] = True

In [18]:
# This gets the indices of the stars that are at valid pixels (this shouldn't filter out many at all).
starIndices = np.where(validPixels[starPixels])[0]

In [19]:
# This tells me which pixels correspond to the values above.
condPix = np.where(validPixels)[0]

# This tells me the pixels for the stars, it will be the same as condPix but have repeats if multiple stars are detected
# on one pixel. By default these will not be sorted.
starPix = starPixels[starIndices]
DETVALIDPIX = DETECTED[starIndices]

# This sorts the pixels. This may seem like an unnecessarily difficult way to do it but it could be useful for future projects
# and it doesn't take much more space or time so I chose to leave it in.
sortInds = starPix.argsort()
starPix = starPix[sortInds[::1]]
DET = DETVALIDPIX[sortInds[::1]]

In [20]:
# These are indices that will be looping through the condPix and starPix arrays in parallel.
condInd = 0
starInd = 0

# This will be used to store the number of stars at each pixel.
detStar = np.zeros_like(condPix) + 1 # Laplace's Rule of Succession
injStar = np.zeros_like(condPix) + 2 # Laplace's Rule of Succession

while starInd < len(starPix):
    if condPix[condInd] == starPix[starInd]: # If the pixels match up in the arrays.
        detStar[condInd] += DET[starInd]
        injStar[condInd] += 1                # Add one to the corresponding spot in the balStar array.
        starInd += 1                         # Add one to the starInd to see if the next index in starPix is also the same.
        # Since the last index of condPix and starPix are the same, starInd will increase the last time through the loop,
        # making this the index that we must restrict in the while loop.
    else:
        condInd += 1 # If the pixels are no longer the same, increase the index you check in the condPix array.

In [21]:
injStar

array([3, 3, 4, ..., 5, 5, 3])

In [22]:
from astropy.table import Table

In [23]:
my_table = Table()
# add columns to the table:
my_table['PIX_2048'] = condPix
# e.g. my_table['pixels'] = pixel_array

In [24]:
my_table['DETECTIONS'] = detStar
my_table['INJECTIONS'] = injStar

In [25]:
my_table.write("/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/valid_pixel_2048_stars.fits", overwrite = True)